In [1]:
import os
import numpy as np
import pandas as pd
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.lunar import LUNAR
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import time

In [2]:

folder_path = './benchad_datasets/'

datasets = []
dataset_names = []
for root, _, filenames in os.walk(folder_path):
    for filename in filenames:
        file_path = os.path.join(root, filename)
        datasets.append(np.load(file_path))
        dataset_names.append(filename.split('_')[1].split('.')[0])

In [3]:
results = pd.DataFrame(columns=['Dataset', '# Samples', '# Features', 'Anomaly Ratio',
                                'ABOD Precision', 'ABOD AUC', 'ABOD Time',
                                'LOF Precision', 'LOF AUC', 'LOF Time',
                                'iForest Precision', 'iForest AUC', 'iForest Time',
                                'AutoEncoder Precision', 'AutoEncoder AUC', 'AutoEncoder Time',
                                'LUNAR Precision', 'LUNAR AUC', 'LUNAR Time'])
algorithms = {
    'ABOD': ABOD,
    'LOF': LOF,
    'iForest': IForest,
    'AutoEncoder': AutoEncoder,
    'LUNAR': LUNAR
}
unsupervised_algorithms = ['ABOD', 'LOF', 'iForest']

In [4]:
def evaluate_unsupervised_algorithm(algorithm, X,  y):
    clf = algorithm(contamination=np.mean(y))

    start_time = time.time()
    clf.fit(X)
    test_scores = clf.decision_function(X)
    end_time = time.time()
    y_pred = clf.predict(X)

    precision = round(precision_score(y, y_pred), ndigits=4)
    auc = round(roc_auc_score(y, test_scores), ndigits=4)
    duration = round(end_time - start_time, ndigits=4)
    return precision, auc, duration

In [5]:
def evaluate_semi_supervised_algorithm(algorithm, X, y):
    if algorithm is AutoEncoder:
        print('AutoEncoder')
        clf = algorithm(
            hidden_neurons=[16, 16, 4, 16, 16], epochs=5, contamination=np.mean(y))
    else:
        clf = algorithm(contamination=np.mean(y))

    X_normal = X[y == 0]
    X_anomaly = X[y == 1]

    random_indices = np.random.choice(
        len(X_normal), size=len(X_anomaly), replace=False)
    array1 = X_normal[random_indices]
    X_train = np.delete(X_normal, random_indices, axis=0)
    X_test = np.concatenate((X_anomaly, array1), axis=0)
    y_test = np.concatenate(
        (np.ones(len(X_anomaly)), np.zeros(len(array1))), axis=0)

    start_time = time.time()
    print(X_train.shape, X_test.shape, y_test.shape)
    clf.fit(X_train)
    test_scores = clf.decision_function(X_test)
    end_time = time.time()
    y_pred = clf.predict(X_test)

    precision = round(precision_score(y_test, y_pred), ndigits=4)
    auc = round(roc_auc_score(y_test, test_scores), ndigits=4)
    duration = round(end_time - start_time, ndigits=4)
    return precision, auc, duration

In [6]:
for i, (dataset, dataset_name) in enumerate(zip(datasets, dataset_names)):
    X = dataset['X']
    y = dataset['y']
    anomaly_ratio = np.mean(y)*100

    for name, algorithm in algorithms.items():
        print(f"Running {name} on {dataset_name}...")
        if name in unsupervised_algorithms:
            precision, auc, exec_time = evaluate_unsupervised_algorithm(
                algorithm, X, y)
        else:
            precision, auc, exec_time = evaluate_semi_supervised_algorithm(
                algorithm, X, y)
        results.loc[i, f'{name} Precision'] = precision
        results.loc[i, f'{name} AUC'] = auc
        results.loc[i, f'{name} Time'] = exec_time

    results.loc[i, 'Dataset'] = dataset_name
    results.loc[i, '# Samples'] = X.shape[0]
    results.loc[i, '# Features'] = X.shape[1]
    results.loc[i, 'Anomaly Ratio'] = anomaly_ratio

Running ABOD on fault...
Running LOF on fault...
Running iForest on fault...
Running AutoEncoder on fault...
AutoEncoder
(595, 27) (1346, 27) (1346,)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 27)                756       
                                                                 
 dropout (Dropout)           (None, 27)                0         
                                                                 
 dense_1 (Dense)             (None, 27)                756       
                                                                 
 dropout_1 (Dropout)         (None, 27)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                448       
                                                                 
 dropout_2 (Dropout)         (None,

c:\Users\matij\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running ABOD on glass...
Running LOF on glass...
Running iForest on glass...
Running AutoEncoder on glass...
AutoEncoder
(196, 7) (18, 7) (18,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 7)                 56        
                                                                 
 dropout_7 (Dropout)         (None, 7)                 0         
                                                                 
 dense_9 (Dense)             (None, 7)                 56        
                                                                 
 dropout_8 (Dropout)         (None, 7)                 0         
                                                                 
 dense_10 (Dense)            (None, 16)                128       
                                                                 
 dropout_9 (Dropout)         (None, 16)   

In [7]:
results.to_csv('results.csv', index=False)
results

,Dataset,# Samples,# Features,Anomaly Ratio,ABOD Precision,ABOD AUC,ABOD Time,LOF Precision,LOF AUC,LOF Time,iForest Precision,iForest AUC,iForest Time,AutoEncoder Precision,AutoEncoder AUC,AutoEncoder Time,LUNAR Precision,LUNAR AUC,LUNAR Time
0,fault,1941,27,34.672849,0.5035,0.6986,1.5692,0.423,0.5957,0.02,0.4264,0.5629,0.1145,0.5749,0.5391,5.2817,0.7883,0.8092,4.2711
1,glass,214,7,4.205607,0.1818,0.845,0.047,0.125,0.8157,0.004,0.1111,0.7626,0.0945,0.5,0.7901,2.7349,0.5,0.9012,1.6424
2,Hepatitis,80,19,16.25,0.1429,0.4788,0.0204,0.3333,0.589,0.012,0.1538,0.7049,0.1091,0.875,0.8047,2.7512,0.6364,0.7515,1.2779
3,InternetAds,1966,1555,18.71821,0.2879,0.6299,5.437,0.4073,0.6451,0.1426,0.462,0.6895,0.1689,0.7879,0.7993,7.1861,0.8924,0.8628,4.9923
4,Ionosphere,351,32,35.897436,0.8092,0.927,0.0753,0.7886,0.8603,0.0199,0.6667,0.8383,0.1125,0.6455,0.8825,2.7592,0.7396,0.9828,1.8464
5,landsat,6435,36,20.714841,0.2198,0.5025,1.6365,0.2778,0.5466,0.116,0.2198,0.491,0.1618,0.5355,0.4587,3.1605,0.8536,0.7902,12.2003
6,mnist,7603,100,9.206892,0.2383,0.7005,6.915,0.2454,0.6449,0.2347,0.2857,0.7885,0.2018,0.8715,0.9087,4.6273,0.9422,0.9333,19.9083
7,musk,3062,166,3.167864,0.0094,0.0528,1.5429,0.0469,0.4124,0.0605,1.0,1.0,0.1309,0.951,1.0,3.2882,0.9798,1.0,9.3589
8,Cardiotocography,2114,21,22.043519,0.2869,0.5553,0.4172,0.3364,0.5965,0.0223,0.4292,0.7172,0.1181,0.747,0.8178,3.0573,0.8517,0.8081,4.6133


# ETRAN Rad


In [3]:
results = pd.DataFrame(columns=['Dataset', '# Samples', '# Features', 'Anomaly Ratio',
                                'iForest Precision', 'iForest AUC', 'iForest Accuracy', 'iForest Recall', 'iForest F1 Score',
                                'iForest Fit Time', 'iForest Score Time',
                                'AutoEncoder Precision', 'AutoEncoder AUC', 'AutoEncoder Accuracy', 'AutoEncoder Recall', 'AutoEncoder F1 Score',
                                'AutoEncoder Fit Time', 'AutoEncoder Score Time',
                                'LUNAR Precision', 'LUNAR AUC', 'LUNAR Accuracy', 'LUNAR Recall', 'LUNAR F1 Score',
                                'LUNAR Fit Time', 'LUNAR Score Time'])
algorithms = {
    'iForest': IForest,
    'AutoEncoder': AutoEncoder,
    'LUNAR': LUNAR
}

In [8]:
def evaluate_semi_supervised_algorithm(algorithm, X, y):
    if algorithm is AutoEncoder:
        print('AutoEncoder')
        clf = algorithm(
            hidden_neurons=[16, 16, 4, 16, 16], epochs=5, contamination=np.mean(y))
    else:
        clf = algorithm(contamination=np.mean(y))

    X_normal = X[y == 0]
    X_anomaly = X[y == 1]

    random_indices = np.random.choice(
        len(X_normal), size=len(X_anomaly), replace=False)
    array1 = X_normal[random_indices]
    X_train = np.delete(X_normal, random_indices, axis=0)
    X_test = np.concatenate((X_anomaly, array1), axis=0)
    y_test = np.concatenate(
        (np.ones(len(X_anomaly)), np.zeros(len(array1))), axis=0)

    print(X_train.shape, X_test.shape, y_test.shape)
    start_time = time.time()
    clf.fit(X_train)
    end_time = time.time()
    fit_time = round(end_time - start_time, ndigits=4)
    test_scores = clf.decision_function(X_test)
    start_time = time.time()
    y_pred = clf.predict(X_test)
    end_time = time.time()
    score_time = round(end_time - start_time, ndigits=4)

    precision = round(precision_score(y_test, y_pred), ndigits=4)
    auc = round(roc_auc_score(y_test, test_scores), ndigits=4)
    accuracy = round(accuracy_score(y_test, y_pred), ndigits=4)
    f1 = round(f1_score(y_test, y_pred), ndigits=4)
    recall = round(recall_score(y_test, y_pred), ndigits=4)

    return precision, auc, accuracy, f1, recall, fit_time, score_time

In [9]:
for i, (dataset, dataset_name) in enumerate(zip(datasets, dataset_names)):
    X = dataset['X']
    y = dataset['y']
    anomaly_ratio = np.mean(y)*100

    for name, algorithm in algorithms.items():
        print(f"Running {name} on {dataset_name}...")
        precision, auc, accuracy, f1, recall, fit_time, score_time = evaluate_semi_supervised_algorithm(
            algorithm, X, y)
        results.loc[i, f'{name} Precision'] = precision
        results.loc[i, f'{name} AUC'] = auc
        results.loc[i, f'{name} Accuracy'] = accuracy
        results.loc[i, f'{name} Recall'] = recall
        results.loc[i, f'{name} F1 Score'] = f1
        results.loc[i, f'{name} Fit Time'] = fit_time
        results.loc[i, f'{name} Score Time'] = score_time

    results.loc[i, 'Dataset'] = dataset_name
    results.loc[i, '# Samples'] = X.shape[0]
    results.loc[i, '# Features'] = X.shape[1]
    results.loc[i, 'Anomaly Ratio'] = anomaly_ratio

Running iForest on fault...
(595, 27) (1346, 27) (1346,)
Running AutoEncoder on fault...
AutoEncoder
(595, 27) (1346, 27) (1346,)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 27)                756       
                                                                 
 dropout (Dropout)           (None, 27)                0         
                                                                 
 dense_1 (Dense)             (None, 27)                756       
                                                                 
 dropout_1 (Dropout)         (None, 27)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                448       
                                                                 
 dropout_2 (Dropout)         (None, 16)                

c:\Users\matij\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running iForest on glass...
(196, 7) (18, 7) (18,)
Running AutoEncoder on glass...
AutoEncoder
(196, 7) (18, 7) (18,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 7)                 56        
                                                                 
 dropout_7 (Dropout)         (None, 7)                 0         
                                                                 
 dense_9 (Dense)             (None, 7)                 56        
                                                                 
 dropout_8 (Dropout)         (None, 7)                 0         
                                                                 
 dense_10 (Dense)            (None, 16)                128       
                                                                 
 dropout_9 (Dropout)         (None, 16)                0         
  

In [10]:
results.to_csv('results.csv', index=False)
results

,Dataset,# Samples,# Features,Anomaly Ratio,iForest Precision,iForest AUC,iForest Accuracy,iForest Recall,iForest F1 Score,iForest Fit Time,...,AutoEncoder F1 Score,AutoEncoder Fit Time,AutoEncoder Score Time,LUNAR Precision,LUNAR AUC,LUNAR Accuracy,LUNAR Recall,LUNAR F1 Score,LUNAR Fit Time,LUNAR Score Time
0,fault,1941,27,34.672849,0.6117,0.6554,0.6048,0.5736,0.592,0.1761,...,0.5058,11.3658,0.1681,0.7927,0.813,0.7036,0.5513,0.6503,8.4339,0.016
1,glass,214,7,4.205607,0.3333,0.7284,0.4444,0.1111,0.1667,0.1755,...,0.2,4.7884,0.0961,0.6667,0.6543,0.7222,0.8889,0.7619,3.1747,0.002
2,Hepatitis,80,19,16.25,0.6667,0.7337,0.6538,0.6154,0.64,0.168,...,0.6087,4.7968,0.0891,0.8889,0.8402,0.7692,0.6154,0.7273,2.3766,0.008
3,InternetAds,1966,1555,18.71821,0.2048,0.3728,0.4334,0.0462,0.0754,0.2195,...,0.6949,9.6393,0.2244,0.8307,0.8628,0.7867,0.7201,0.7715,9.1712,0.0961
4,Ionosphere,351,32,35.897436,0.7391,0.9235,0.8056,0.9444,0.8293,0.1681,...,0.7986,4.6974,0.129,0.8214,0.9567,0.8571,0.9127,0.8647,2.618,0.0
5,landsat,6435,36,20.714841,0.6445,0.6172,0.5833,0.3713,0.4712,0.2399,...,0.344,5.4995,0.2609,0.8215,0.7882,0.6999,0.5109,0.63,22.6455,0.0564
6,letter,1600,32,6.25,0.5263,0.5897,0.505,0.1,0.1681,0.1996,...,0.1239,5.3937,0.1191,0.9123,0.9298,0.735,0.52,0.6624,9.7978,0.008
7,magic,19020,10,35.162986,0.65,0.748,0.6617,0.7008,0.6744,0.2561,...,0.6304,6.5919,0.8132,0.8209,0.8473,0.7681,0.6859,0.7473,33.972,1.1786
8,mammography,11183,6,2.324958,0.9391,0.8978,0.6942,0.4154,0.576,0.3203,...,0.601,8.9595,0.1271,0.9725,0.8906,0.6981,0.4077,0.5745,61.8515,0.029
9,mnist,7603,100,9.206892,0.8488,0.8487,0.7043,0.4971,0.627,0.3084,...,0.8087,7.1532,0.1759,0.9065,0.932,0.8479,0.7757,0.836,36.2766,0.048


In [7]:
from sklearn.model_selection import train_test_split


def evaluate_semi_supervised_algorithm_stratified(algorithm, X, y, test_size=0.3):
    if algorithm is AutoEncoder:
        clf = algorithm(
            hidden_neurons=[16, 16, 4, 16, 16], epochs=25, contamination=np.mean(y))
    else:
        clf = algorithm(contamination=np.mean(y))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y)
    X_train_normal = X_train[y_train == 0]
    print(y.mean(), y_train.mean(), y_test.mean())

    start_time = time.time()
    clf.fit(X_train_normal)
    end_time = time.time()
    fit_time = round(end_time - start_time, ndigits=4)
    test_scores = clf.decision_function(X_test)
    start_time = time.time()
    y_pred = clf.predict(X_test)
    end_time = time.time()
    score_time = round(end_time - start_time, ndigits=4)

    precision = round(precision_score(y_test, y_pred), ndigits=4)
    auc = round(roc_auc_score(y_test, test_scores), ndigits=4)
    accuracy = round(accuracy_score(y_test, y_pred), ndigits=4)
    f1 = round(f1_score(y_test, y_pred), ndigits=4)
    recall = round(recall_score(y_test, y_pred), ndigits=4)

    return precision, auc, accuracy, f1, recall, fit_time, score_time

In [8]:
for i, (dataset, dataset_name) in enumerate(zip(datasets, dataset_names)):
    X = dataset['X']
    y = dataset['y']
    anomaly_ratio = np.mean(y)*100

    for name, algorithm in algorithms.items():
        print(f"Running {name} on {dataset_name}...")
        precision, auc, accuracy, f1, recall, fit_time, score_time =\
            evaluate_semi_supervised_algorithm_stratified(
                algorithm, X, y)
        results.loc[i, f'{name} Precision'] = precision
        results.loc[i, f'{name} AUC'] = auc
        results.loc[i, f'{name} Accuracy'] = accuracy
        results.loc[i, f'{name} Recall'] = recall
        results.loc[i, f'{name} F1 Score'] = f1
        results.loc[i, f'{name} Fit Time'] = fit_time
        results.loc[i, f'{name} Score Time'] = score_time

    results.loc[i, 'Dataset'] = dataset_name
    results.loc[i, '# Samples'] = X.shape[0]
    results.loc[i, '# Features'] = X.shape[1]
    results.loc[i, 'Anomaly Ratio'] = anomaly_ratio

results.to_csv('results_stratified_25epoch.csv', index=False)
results

Running iForest on fault...
0.3467284904688305 0.34683357879234167 0.346483704974271
Running AutoEncoder on fault...
0.3467284904688305 0.34683357879234167 0.346483704974271
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_160 (Dense)           (None, 27)                756       
                                                                 
 dropout_140 (Dropout)       (None, 27)                0         
                                                                 
 dense_161 (Dense)           (None, 27)                756       
                                                                 
 dropout_141 (Dropout)       (None, 27)                0         
                                                                 
 dense_162 (Dense)           (None, 16)                448       
                                                                 
 dropout_14

c:\Users\matij\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


None
Epoch 1/25
2/2 [==============================] - 2s 252ms/step - loss: 3.9176 - val_loss: 2.3629
Epoch 2/25
2/2 [==============================] - 0s 28ms/step - loss: 3.9947 - val_loss: 2.3280
Epoch 3/25
2/2 [==============================] - 0s 28ms/step - loss: 3.6589 - val_loss: 2.2967
Epoch 4/25
2/2 [==============================] - 0s 26ms/step - loss: 3.5921 - val_loss: 2.2691
Epoch 5/25
2/2 [==============================] - 0s 25ms/step - loss: 3.5582 - val_loss: 2.2438
Epoch 6/25
2/2 [==============================] - 0s 27ms/step - loss: 3.4937 - val_loss: 2.2209
Epoch 7/25
2/2 [==============================] - 0s 27ms/step - loss: 3.4408 - val_loss: 2.1998
Epoch 8/25
2/2 [==============================] - 0s 26ms/step - loss: 3.3575 - val_loss: 2.1796
Epoch 9/25
2/2 [==============================] - 0s 25ms/step - loss: 3.3038 - val_loss: 2.1603
Epoch 10/25
2/2 [==============================] - 0s 26ms/step - loss: 3.1843 - val_loss: 2.1423
Epoch 11/25
2/2 [======

,Dataset,# Samples,# Features,Anomaly Ratio,iForest Precision,iForest AUC,iForest Accuracy,iForest Recall,iForest F1 Score,iForest Fit Time,...,AutoEncoder F1 Score,AutoEncoder Fit Time,AutoEncoder Score Time,LUNAR Precision,LUNAR AUC,LUNAR Accuracy,LUNAR Recall,LUNAR F1 Score,LUNAR Fit Time,LUNAR Score Time
0,fault,1941,27,34.672849,0.4815,0.6933,0.6364,0.6436,0.5508,0.0945,...,0.4623,4.5276,0.0739,0.6224,0.8103,0.7358,0.604,0.6131,3.7967,0.0205
1,glass,214,7,4.205607,0.0,0.7527,0.9231,0.0,0.0,0.1025,...,0.0,2.8452,0.058,0.0,0.8118,0.8923,0.0,0.0,1.7724,0.001
2,Hepatitis,80,19,16.25,0.0,0.85,0.8333,0.0,0.0,0.0784,...,0.4444,3.6008,0.0576,0.0,0.7,0.7917,0.0,0.0,1.5849,0.0062
3,InternetAds,1966,1555,18.71821,0.23,0.512,0.722,0.2091,0.219,0.1215,...,0.4695,23.5302,0.1247,0.6458,0.8222,0.861,0.5636,0.6019,4.6549,0.0515
4,Ionosphere,351,32,35.897436,0.5645,0.8862,0.717,0.9211,0.7,0.1126,...,0.7048,3.0276,0.0614,0.6552,0.9834,0.8113,1.0,0.7917,1.5153,0.0071
5,landsat,6435,36,20.714841,0.3263,0.5947,0.7074,0.3875,0.3543,0.156,...,0.2236,7.5646,0.1201,0.5312,0.7902,0.8058,0.5325,0.5318,11.4915,0.0244
6,letter,1600,32,6.25,0.0682,0.6228,0.8583,0.1,0.0811,0.1237,...,0.0333,4.0131,0.0723,0.5556,0.9334,0.9417,0.3333,0.4167,4.1365,0.004
7,magic,19020,10,35.162986,0.5162,0.762,0.6646,0.7298,0.6047,0.1902,...,0.5401,12.8929,0.2277,0.6811,0.8589,0.7878,0.7453,0.7117,28.2592,0.3528
8,mammography,11183,6,2.324958,0.2843,0.8774,0.9636,0.3718,0.3222,0.1504,...,0.2919,11.1621,0.1704,0.4028,0.8728,0.9726,0.3718,0.3867,32.4583,0.1239
9,mnist,7603,100,9.206892,0.3742,0.8709,0.8746,0.5381,0.4414,0.2438,...,0.5557,12.9826,0.1771,0.6182,0.9279,0.9307,0.6476,0.6326,23.2555,0.056
